In [1]:
import keras
import numpy as np
import scipy.io
from pathlib import Path
import os

Using TensorFlow backend.


## Open labels

In [2]:
path = 'C:\\Users\\Rodolfo\\Documents\\algoritmos-git\\custom_cnn\\Data\\Processados\\'

In [3]:
y_train = np.load(Path(path,'y_train.npy'))
y_test = np.load(Path(path, 'y_test.npy'))
y_train = np.expand_dims(y_train, axis=1)
y_test = np.expand_dims(y_test, axis=1)

In [4]:
from keras.utils import np_utils
# one-hot encode the labels
num_classes = len(np.unique(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
y_train

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [6]:
y_test

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

# Open data

In [7]:
path = Path(path, '28_28')
X_train_1 = np.load(Path(path, 'X_train_1.npy'))
X_train_2 = np.load(Path(path, 'X_train_2.npy'))
X_test_1 = np.load(Path(path, 'X_test_1.npy'))
X_test_2 = np.load(Path(path, 'X_test_2.npy'))
print(X_train_1.shape)
print(X_train_2.shape)
print(X_test_1.shape)
print(X_test_2.shape)

(2200, 28, 28)
(2200, 28, 28)
(1000, 28, 28)
(1000, 28, 28)


### Shape desejado: x_train shape: (45000, 32, 32, 3) -> n_obs, n_row, n_col, n_chan

In [8]:
n_obs, n_row, n_col = X_train_1.shape
n_chan=2
X_train = np.zeros((n_obs, n_row, n_col, n_chan))
X_train.shape

(2200, 28, 28, 2)

In [9]:
X_train[:,:,:,0]=X_train_1
X_train[:,:,:,1]=X_train_2
X_train.shape

(2200, 28, 28, 2)

In [10]:
n_obs, n_row, n_col = X_test_1.shape
n_chan=2
X_test = np.zeros((n_obs, n_row, n_col, n_chan))
X_test.shape

(1000, 28, 28, 2)

In [11]:
X_test[:,:,:,0]=X_test_1
X_test[:,:,:,1]=X_test_2
X_test.shape

(1000, 28, 28, 2)

# Shuffle X_train

In [12]:
arr = np.arange(y_train.shape[0])
np.random.shuffle(arr)
X_train = X_train[arr,:,:,:]
y_train = y_train[arr]

### 3. Rescale the Images by Dividing Every Pixel in Every Image by 255

In [13]:
# rescale [0,255] --> [0,1]
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

### 4.  Break Dataset into Training, Testing, and Validation Sets

In [14]:
# break training set into training and validation sets
(X_train, X_valid) = X_train[:1800], X_train[1800:]
(y_train, y_valid) = y_train[:1800], y_train[1800:]

# print shape of training set
print('x_train shape:', X_train.shape)

# print number of training, validation, and test images
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(X_valid.shape[0], 'validation samples')

x_train shape: (1800, 28, 28, 2)
1800 train samples
1000 test samples
400 validation samples


In [15]:
n_obs, n_row, n_col, n_chan = X_train.shape

In [16]:
print(n_obs, n_row, n_col, n_chan)

1800 28 28 2


### 5. Define the Model Architecture 

In [22]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# "valid" means no padding. "same" results in padding
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='valid', activation='relu', strides=(1,1), input_shape=(n_row, n_col, n_chan)))
model.add(MaxPooling2D(pool_size=2, strides=(1,1)))
model.add(Conv2D(filters=32, kernel_size=2, padding='valid', activation='relu', strides=(1,1)))
model.add(MaxPooling2D(pool_size=2,strides=2))
model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 27, 27, 16)        144       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 26, 26, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 25, 25, 32)        2080      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               921800    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
Total para

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# "valid" means no padding. "same" results in padding
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='valid', activation='relu', strides=(1,1), input_shape=(n_row, n_col, n_chan)))
model.add(MaxPooling2D(pool_size=2, strides=(1,1)))
model.add(Conv2D(filters=32, kernel_size=2, padding='valid', activation='relu', strides=(1,1)))
model.add(MaxPooling2D(pool_size=2,strides=2)

'''
model.add(Conv2D(filters=64, kernel_size=2, padding='valid', activation='relu', strides=(1,1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=2, padding='valid', activation='relu', strides=(1,1)))
model.add(MaxPooling2D(pool_size=2))'''
#model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(200, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='sigmoid'))

model.summary()

### 6. Compile the Model 

In [23]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

### 7. Train the Model 

In [24]:
model_path = 'model_15.weights.best.hdf5'

In [26]:
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath=model_path, verbose=1, 
                               save_best_only=True)
hist = model.fit(X_train, y_train, batch_size=50, epochs=100,
          validation_data=(X_valid, y_valid), callbacks=[checkpointer], 
          verbose=2, shuffle=True)

Train on 1800 samples, validate on 400 samples
Epoch 1/100
 - 3s - loss: 0.1472 - acc: 0.9517 - val_loss: 0.8471 - val_acc: 0.6650

Epoch 00001: val_loss improved from inf to 0.84705, saving model to model_15.weights.best.hdf5
Epoch 2/100
 - 3s - loss: 0.1433 - acc: 0.9494 - val_loss: 0.8321 - val_acc: 0.6900

Epoch 00002: val_loss improved from 0.84705 to 0.83213, saving model to model_15.weights.best.hdf5
Epoch 3/100
 - 3s - loss: 0.1454 - acc: 0.9494 - val_loss: 0.8307 - val_acc: 0.6850

Epoch 00003: val_loss improved from 0.83213 to 0.83071, saving model to model_15.weights.best.hdf5
Epoch 4/100
 - 4s - loss: 0.1071 - acc: 0.9711 - val_loss: 1.0492 - val_acc: 0.6600

Epoch 00004: val_loss did not improve from 0.83071
Epoch 5/100
 - 4s - loss: 0.1084 - acc: 0.9683 - val_loss: 0.9585 - val_acc: 0.6675

Epoch 00005: val_loss did not improve from 0.83071
Epoch 6/100
 - 3s - loss: 0.0972 - acc: 0.9750 - val_loss: 1.0781 - val_acc: 0.6750

Epoch 00006: val_loss did not improve from 0.830

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:436: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current, self.best):


 - 3s - loss: nan - acc: 0.5744 - val_loss: nan - val_acc: 0.5225

Epoch 00037: val_loss did not improve from 0.83071
Epoch 38/100
 - 3s - loss: nan - acc: 0.4950 - val_loss: nan - val_acc: 0.5225

Epoch 00038: val_loss did not improve from 0.83071
Epoch 39/100
 - 3s - loss: nan - acc: 0.4950 - val_loss: nan - val_acc: 0.5225

Epoch 00039: val_loss did not improve from 0.83071
Epoch 40/100
 - 3s - loss: nan - acc: 0.4950 - val_loss: nan - val_acc: 0.5225

Epoch 00040: val_loss did not improve from 0.83071
Epoch 41/100
 - 3s - loss: nan - acc: 0.4950 - val_loss: nan - val_acc: 0.5225

Epoch 00041: val_loss did not improve from 0.83071
Epoch 42/100
 - 3s - loss: nan - acc: 0.4950 - val_loss: nan - val_acc: 0.5225

Epoch 00042: val_loss did not improve from 0.83071
Epoch 43/100
 - 3s - loss: nan - acc: 0.4950 - val_loss: nan - val_acc: 0.5225

Epoch 00043: val_loss did not improve from 0.83071
Epoch 44/100
 - 3s - loss: nan - acc: 0.4950 - val_loss: nan - val_acc: 0.5225

Epoch 00044: val

 - 3s - loss: nan - acc: 0.4950 - val_loss: nan - val_acc: 0.5225

Epoch 00100: val_loss did not improve from 0.83071


In [27]:
model_path

'model_15.weights.best.hdf5'

### 8. Load the Model with the Best Validation Accuracy

In [28]:
# load the weights that yielded the best validation accuracy
model.load_weights(model_path)

### 9. Calculate Classification Accuracy on Test Set

In [29]:
# evaluate and print test accuracy
score = model.evaluate(X_test, y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.677


In [30]:
model.predict(X_test)

array([[2.5439262e-04, 1.1388659e-03],
       [1.8231273e-03, 2.5123358e-05],
       [1.6225874e-03, 1.2640059e-03],
       ...,
       [1.5443563e-04, 6.8783462e-03],
       [4.2042196e-02, 2.2790134e-03],
       [4.3201149e-03, 1.3833761e-02]], dtype=float32)

In [31]:
model_path

'model_15.weights.best.hdf5'